In [2]:
!git clone https://github.com/kesaroid/Glaucoma-Detection.git

Cloning into 'Glaucoma-Detection'...
remote: Enumerating objects: 1008, done.
remote: Total 1008 (delta 0), reused 0 (delta 0), pack-reused 1008
Receiving objects: 100% (1008/1008), 47.97 MiB | 44.77 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [0]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import BatchNormalization, Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from imgaug import augmenters as iaa

img_width, img_height = 256, 256
input_shape = (img_width, img_height, 3)

train_data_dir = "data/train"
validation_data_dir = "data/validation"
nb_train_samples =100
nb_validation_samples = 100
batch_size = 16
epochs = 100

input = Input(shape=input_shape)

block1 = BatchNormalization(name='norm_0')(input)

# Block 1
block1 = Conv2D(8, (3,3), name='conv_11', activation='relu')(block1)
block1 = Conv2D(16, (3,3), name='conv_12', activation='relu')(block1)
block1 = Conv2D(32, (3,3), name='conv_13', activation='relu')(block1)
block1 = Conv2D(64, (3,3), name='conv_14', activation='relu')(block1)
block1 = MaxPooling2D(pool_size=(2, 2))(block1)
block1 = BatchNormalization(name='norm_1')(block1)

block1 = Conv2D(16, 1)(block1)

# Block 2
block2 = Conv2D(32, (3,3), name='conv_21', activation='relu')(block1)
block2 = Conv2D(64, (3,3), name='conv_22', activation='relu')(block2)
block2 = Conv2D(64, (3,3), name='conv_23', activation='relu')(block2)
block2 = Conv2D(128, (3,3), name='conv_24', activation='relu')(block2)
block2 = MaxPooling2D(pool_size=(2, 2))(block2)
block2 = BatchNormalization(name='norm_2')(block2)

block2 = Conv2D(64, 1)(block2)

# Block 3
block3 = Conv2D(64, (3,3), name='conv_31', activation='relu')(block2)
block3 = Conv2D(128, (3,3), name='conv_32', activation='relu')(block3)
block3 = Conv2D(128, (3,3), name='conv_33', activation='relu')(block3)
block3 = Conv2D(64, (3,3), name='conv_34', activation='relu')(block3)
block3 = MaxPooling2D(pool_size=(2, 2))(block3)
block3 = BatchNormalization(name='norm_3')(block3)

# Block 4
block4 = Conv2D(64, (3,3), name='conv_41', activation='relu')(block3)
block4 = Conv2D(32, (3,3), name='conv_42', activation='relu')(block4)
block4 = Conv2D(16, (3,3), name='conv_43', activation='relu')(block4)
block4 = Conv2D(8, (2,2), name='conv_44', activation='relu')(block4)
block4 = MaxPooling2D(pool_size=(2, 2))(block4)
block4 = BatchNormalization(name='norm_4')(block4)

block4 = Conv2D(2, 1)(block4)

block5 = GlobalAveragePooling2D()(block4)
output = Activation('softmax')(block5)

model = Model(inputs=[input], outputs=[output])
model.summary()
model.compile(loss="categorical_crossentropy", optimizer=optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])

# Initiate the train and test generators with data Augumentation
sometimes = lambda aug: iaa.Sometimes(0.6, aug)
seq = iaa.Sequential([
                      iaa.GaussianBlur(sigma=(0 , 1.0)),
                      iaa.Sharpen(alpha=1, lightness=0),
                      iaa.CoarseDropout(p=0.1, size_percent=0.15),
                              sometimes(iaa.Affine(
                                                    scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
                                                    translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
                                                    rotate=(-30, 30),
                                                    shear=(-16, 16)))
                    ])


train_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=seq.augment_image,
    horizontal_flip=True,
    vertical_flip=True)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode="categorical")

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    class_mode="categorical")

checkpoint = ModelCheckpoint("f1.h5", monitor='acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, verbose=0, mode='auto', cooldown=0, min_lr=0)

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[checkpoint, reduce_lr]
)

In [0]:
from tkinter import *
from tkinter import messagebox
from keras.models import load_model
import matplotlib.pyplot as plt
import cv2
import numpy as np
import os
import pandas


model = load_model('f1.h5')


def get_filenames():
    global path
    path = r"test"
    return os.listdir(path)


def curselect(event):
    global spath
    index = t1.curselection()[0]
    spath = t1.get(index)
    return(spath)


def autoroi(img):

    gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    thresh = cv2.threshold(gray_img, 130, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.dilate(thresh, None, iterations=5)

    contours, hierarchy = cv2.findContours(
        thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    biggest = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(biggest)
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
    roi = img[y:y+h, x:x+w]

    return roi


def prediction():

    img = cv2.imread('test/%s' % (spath))
    img = autoroi(img)
    img = cv2.resize(img, (256, 256))
    img = np.reshape(img, [1, 256, 256, 3])

    prob = model.predict(img)
    Class = prob.argmax(axis=-1)

    return(Class)


def run():

    Class = prediction()
    if (Class == 0):
        messagebox.showinfo('Prediction', 'You have been diagnosed with Glaucoma')
    else:
        messagebox.showinfo('Prediction', 'Congratulations! You are Healthy')


def run_all():

    x = os.listdir(path)
    y = []
    affected = 0

    for i in x:
        img = cv2.imread('test/%s' % (i))
        img = autoroi(img)
        img = cv2.resize(img, (256, 256))
        img = np.reshape(img, [1, 256, 256, 3])

        prob = model.predict(img)
        Class = prob.argmax(axis=-1)
        y.append(Class[0])
        if Class == 1:
            affected += 1

    df = pandas.DataFrame(data=y, index=x, columns=["output"])
    df.to_csv('output.csv', sep=',')


def ROI():
    img = cv2.imread('test/%s' % (spath))
    roi = autoroi(img)
    cv2.imshow("Region of Interest", roi)


def preview():
    img = cv2.imread('test/%s' % (spath))
    cv2.imshow('Image', img)


def graph():

    total = len(os.listdir(path))
    affected = pandas.read_csv('output.csv')
    affected = affected['output'].sum()

    healthy = total - affected

    piey = ["Glaucomatous", "Healthy"]
    piex = [affected, healthy]

    plt.axis("equal")
    plt.pie(piex, labels=piey, radius=1.5, autopct='%0.1f%%', explode=[0.2, 0])
    plt.show()

# Frontend GUI


window = Tk()
window.title("Glaucoma Detection")
window.geometry('1000x550')
window.configure(background='grey')

l1 = Label(window, text="Test Image", font=("Arial", 20), padx=10, bg='grey')
l1.grid(row=0, column=0)

b1 = Button(window, text='Run', font=("Arial", 20), command=run)
b1.grid(row=1, column=3)

b2 = Button(window, text='Preview', font=("Arial", 20), command=preview)
b2.grid(row=1, column=2, rowspan=2, padx=10)

b2 = Button(window, text='ROI', font=("Arial", 20), command=ROI)
b2.grid(row=2, column=2, rowspan=3, padx=10)

b3 = Button(window, text='Run all', font=("Arial", 20), command=run_all)
b3.grid(row=2, column=3)

b4 = Button(window, text='Graph', font=("Arial", 20), command=graph)
b4.grid(row=3, column=3)

t1 = Listbox(window, height=20, width=60, selectmode=SINGLE, font=("Arial", 15), justify=CENTER)
t1.grid(row=1, column=0, rowspan=3, padx=10)
for filename in get_filenames():
    t1.insert(END, filename)
t1.bind('<<ListboxSelect>>', curselect)

sb1 = Scrollbar(window)
sb1.grid(row=1, column=1, rowspan=4)

t1.configure(yscrollcommand=sb1.set)
sb1.configure(command=t1.yview)


window.mainloop()